In [ ]:
#----------------------------------------import----------------------------------------
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd

import matplotlib.pyplot as plt

np.random.seed(7)

#--------------------------------------- constant ---------------------------------------
# 이미지의 크기 및 모양
img_rows = 28
img_cols = 28

batch_size = 128 # 뉴런의 개수
num_classes = 10 # 카데고리 개수
epochs = 12      # 학습 횟수

#------------------------------------ MNIST dataset ------------------------------------
# train : 6만개의 트레이닝 데이터
# test : 신경망(nn)이 지금까지 본 적 없는 1만개의 데이터, 학습검증용
# x : 학습데이터로, 28 * 28사이즈의 2차원 리스트 ( 이미지 )
# y : 정답으로, 0~9까지의 카테고리
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
A = x_test
B = y_test
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(60000, 28, 28) (60000,)
(10000, 28, 28) (10000,)


In [ ]:
#------------------------------------데이터 정제------------------------------------------
# 신경망(nn)이 이해할 수 있게 변환해줘야만 한다.
# 1). x : 6만개의 2차원 배열로 된 인풋 이미지, 케라스의 nn 이 받아들일 수 있게 4차원 배열로 바꿔줘야 한다.  (=reshape)
input_shape = (img_rows, img_cols, 1)
x_train = x_train.reshape(60000, img_rows, img_cols, 1)
x_test = x_test.reshape(10000, img_rows, img_cols, 1)

x_train = x_train.astype('float32') / 255. # 이미지 데이터는 28 * 28 배열로 0~255사이의 값으로 되어 있으므로 0~1사이의 값으로 만들어야 하기 때문에 255로 나눈다.
x_test = x_test.astype('float32') / 255.

# 2). y : x에 대한 답으로, 0~9까지의 숫자이다. nn 이 이해할 수 있도록 one-hot-encoding 으로 바꿔줘야한다. (=난 get_dummies로 할거야)
y_train = pd.get_dummies(y_train)
y_test = pd.get_dummies(y_test)
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(60000, 28, 28, 1) (60000, 10)
(10000, 28, 28, 1) (10000, 10)


In [ ]:
# ---------------------------------------model---------------------------------------------
model = keras.Sequential()
# Convolution : 이미지의 특징 검출, 데이터양이 크다.
# MaxPooling : 정해진 구간(ex. 2*x) 안에서 가장 큰 값만 남기고 나머지는 버리는 방식, 데이터양이 획기적으로 줄어든다.
# Layer 1
model.add(keras.layers.Conv2D(32, kernel_size=5, padding='same', activation='swish'))
model.add(keras.layers.MaxPooling2D())
# Layer 2
model.add(keras.layers.Conv2D(64, kernel_size=5, padding='same', activation='swish'))
model.add(keras.layers.MaxPooling2D())

# dropout: 레이어가 많아 발생할 수 있는 과적합을 방지하기 위해 학습데이터의 0.25% 는 버린다.
model.add(keras.layers.Dropout(0.25))
# flatten: 2차원 데이터를 1차원으로 바꾼다.
model.add(keras.layers.Flatten())

# fully connected node 인풋 뉴런들이 relu로 들어옴.
model.add(keras.layers.Dense(1000, activation='swish'))
model.add(keras.layers.Dropout(0.5))

model.add(keras.layers.Dense(num_classes, activation='softmax'))


model.compile(loss='categorical_crossentropy', metrics='accuracy')

In [ ]:
# 모델을 학습한다.
model.fit(x_train, y_train, epochs=epochs)

Epoch 1/12
1875/1875 [==============================] - 9s 5ms/step - loss: 0.2410 - accuracy: 0.9251
Epoch 2/12
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0692 - accuracy: 0.9803
Epoch 3/12
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0530 - accuracy: 0.9851
Epoch 4/12
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0427 - accuracy: 0.9880
Epoch 5/12
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0365 - accuracy: 0.9898
Epoch 6/12
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0291 - accuracy: 0.9917
Epoch 7/12
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0301 - accuracy: 0.9914
Epoch 8/12
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0268 - accuracy: 0.9937
Epoch 9/12
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0241 - accuracy: 0.9938
Epoch 10/12
1875/1875 [==============================] - 9s 5ms/step - loss: 0.024

In [ ]:
print(B[0:10])
plt.imshow(x_test[2].reshape(img_rows, img_cols), cmap='gray')
plt.show()
print(x_test[2].shape)
print(x_test[2])

In [ ]:
pred = model.predict_classes(x_test[0].reshape(1, img_rows, img_cols, 1))
print(pred)

[7]


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [ ]:
y_test[0:5]

,0,1,2,3,4,5,6,7,8,9
0,0,0,0,0,0,0,0,1,0,0
1,0,0,1,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0


In [ ]:
model.save("MNIST_CNN_DATA.h5")